## 【实验8-1】调用requests库和BeautifulSoup库，使用BeautifulSoup库爬取计科院网站首页动态新闻
爬取目标网址：https://www.swpu.edu.cn/scs/
调用requests库和BeautifulSoup库，西南石油大学计算机科学学院首页的爬取路径为：
path='https://www.swpu.edu.cn/scs/index.htm' 
相关注释如下：
> 调用requests库 
> 调用BeautifulSoup库
> 返回一个response对象，赋值给res 
> 把res的内容以字符串的形式返回 
> 把网页解析为BeautifulSoup对象 
> 通过定位标签和属性提取我们想要的数据  
> 打印输出爬取结果

通过以下PowerShell使用pip安装requests和BeautifulSoup库。

In [3]:
!powershell pip install requests beautifulsoup4 -i https://pypi.mirrors.ustc.edu.cn/simple/

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [112]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.swpu.edu.cn/scs/index.htm'
res = requests.get(url)
res.encoding = res.apparent_encoding
html_content = res.text
soup = BeautifulSoup(html_content, 'html.parser',from_encoding='utf-8')
# 定位包含动态新闻的<ul>标签
ul_tag = soup.find('ul', class_='dynamic_list')

# 清空out。txt
with open('out.txt','w',encoding='utf-8') as f:
    pass

# 如果找到了<ul>标签
if ul_tag:
    # 遍历<ul>下的每一个<li>标签
    for li_tag in ul_tag.find_all('li'):
        # 在每个<li>标签中，遍历每一个<a>标签
        for a_tag in li_tag.find_all('a'):
            s:str=a_tag.text
            print(s)
            with open('out.txt','a',encoding='utf-8') as f:
                f.write(s+'\n')
else:
    print("未找到动态新闻列表")


[04-24]天津科技大学人工智能学院来我院调研
[04-22]学院党委开展“追忆革命先烈，赓续红色血脉
[04-22]赵刚同志任计算机与软件学院院长、党委副书
[04-17]计算机与软件学院召开2024届毕业生就业工作
[04-16]我院学生团队斩获ASC世界大学生超级计算机
[04-15]马克思主义学院来计算机与软件学院调研虚拟


【实验8-2】调用requests库和os库（创建文件夹），在网上爬取一个图片文件并使用保存在本地D盘picture文件夹中。如果文件不存在，就创建并保存，如果文件存在，就提示已经存在，如果爬取不成功，就提示爬取失败。

In [114]:
import os
import requests

# 图片保存路径
save_dir = 'D:/picture'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

image_url = 'https://www.swpu.edu.cn/scs/img/welcome.jpg'

filename = os.path.join(save_dir, '爬取到的图片.jpg')

if os.path.exists(filename):
    print("图片已存在")
else:
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
            print("图片保存成功")
        else:
            print("爬取失败，状态码：", response.status_code)
    except Exception as e:
        print("爬取失败：", e)


图片保存成功


【实验8-3】目标：从下厨房网站爬取本周热门菜谱清单（菜品名称、网址、食材构成）
爬取目标网址：http://www.xiachufang.com
(1)阅读网站Robots协议：http://www.xiachufang.com/robots.txt 识别Disallow和Allow: /category/*/?ref=*
    已阅读，explore可以爬取。
(2)计划爬取到的信息：菜名、食材、食材对应的详细页URL
(3)锁定目标、分析HTML结构、查找信息
(4)代码实现过程如下：
Step 1. 选取的URL是： http://www.xiachufang.com/explore/ 用requests.get()来获取数据。
Step 2. 使用BeautifulSoup来解析
Step 3. 提取最小父级标签。它的标签是<div>，有一个class属性，其值是info pure-u。可以使用find_all()语法，来找到它们。
Step 4. 不先急于提取出所有的菜名、URL和食材。先尝试提取一组，等成功了，再去写循环提取所有。
Step 5. 写循环存列表

In [113]:
import requests
from bs4 import BeautifulSoup

# Step 1: 获取数据
url = 'http://www.xiachufang.com/explore/'
response = requests.get(url)
html_content = response.text

# Step 2: 使用BeautifulSoup解析
soup:BeautifulSoup = BeautifulSoup(html_content, 'html.parser')

# Step 3: 提取最小父级标签
recipe_cards:[] = soup.find_all('div', class_='info pure-u')

success:bool = False

try:
    # Step 4: 提取一组信息
    recipe = recipe_cards[0]
    success = True
except:
    print('提取信息失败')

if success:
    # Step 5: 循环提取所有信息
    recipes_data = []
    
    for recipe in recipe_cards:
        # 菜名
        name = recipe.find('p', class_='name').text.strip()
        
        # 菜谱详情页URL
        recipe_url = 'http://www.xiachufang.com' + recipe.find('p', class_='name').a['href']
        
        # 食材
        ingredients = recipe.find('p', class_='ing ellipsis').text.strip()
        
        recipes_data.append({
            'name': name,
            'recipe_url': recipe_url,
            'ingredients': ingredients
        })
    
    # 打印结果
    for recipe_data in recipes_data:
        print("菜名:", recipe_data['name'])
        print("食材:", recipe_data['ingredients'])
        print("详情页URL:", recipe_data['recipe_url'])
        print("\n")


菜名: 万能的“拌面，水饺🥟”调料
食材: 蒜末、盐、蚝油、香油、葱、鸡精、生抽、糖
详情页URL: http://www.xiachufang.com/recipe/107312845/


菜名: 超开胃的番茄土豆肥牛汤‼️下米饭无敌了
食材: 土豆、番茄、肥牛、大蒜、小葱
详情页URL: http://www.xiachufang.com/recipe/105910837/


菜名: ❗拿肉都不换的手撕包菜，5分钟快手家常菜🔥
食材: 包菜、大蒜、干辣椒
详情页URL: http://www.xiachufang.com/recipe/106397717/


菜名: 比布丁还要嫩，老少皆宜的肉沫蒸蛋
食材: 鸡蛋、肉沫、葱花、蚝油、生抽、盐、淀粉水
详情页URL: http://www.xiachufang.com/recipe/106781050/


菜名: 芒果脆皮雪糕丨锁死这个配方
食材: 芒果肉、原味酸奶、淡奶油、糖、梦龙脆皮、黑巧克力、花生碎、椰子油
详情页URL: http://www.xiachufang.com/recipe/107136340/


菜名: 青椒火腿炒鸡蛋
食材: 青椒、火腿肠、鸡蛋、蒜末、生抽、蚝油
详情页URL: http://www.xiachufang.com/recipe/106886517/


菜名: ‼️比辣条还好吃百倍〰️香辣豆皮✅零厨艺
食材: 千张（豆皮）、小米椒、、葱花、香菜、生抽、香醋、蚝油、盐糖
详情页URL: http://www.xiachufang.com/recipe/106030102/


菜名: 童年最爱.奶油小布丁
食材: 牛奶、淡奶油、细砂糖、奶粉、玉米淀粉
详情页URL: http://www.xiachufang.com/recipe/107127593/


菜名: 1分钟搞定好喝燕麦奶昔！减脂掉秤|低卡饱腹！
食材: 即食燕麦、牛奶、水果
详情页URL: http://www.xiachufang.com/recipe/106903165/


菜名: 糖醋排骨（试验多次的最佳口感）
食材: 排骨、醋、冰糖/白糖、料酒、生抽、老抽、大料、白芝麻、淀粉加水
详情页URL: http://www.xiachufang.com/recipe/105979376/


菜名:

【实验8-4】目标：爬取豆瓣Top250电影的信息（包括：序号/电影名/评分/推荐语/链接）
目标网址：https://movie.douban.com/top250
采取的步骤如下：
Step 1. 选取的URL是https://movie.douban.com/top250?start=0&filter=
，用requests.get()来获取数据。
Step 2. 使用BeautifulSoup来解析
Step 3. 提取最小父级标签。它的标签是<div>，有一个class属性，其值是info pure-u。可以使用find_all()语法，来找到它们。
Step 4. 不先急于提取出所有的信息。先尝试提取一组，等成功了，再去写循环提取所有。
Step 5. 写循环存列表

In [115]:
import requests
from bs4 import BeautifulSoup
class MovieInfo:
    def __init__(self,rank: str,name: str,rating: float,quote: str,movie_url: str):
        self.rank: str = str(rank)
        self.name: str = str(name)
        self.rating: float = float(rating)
        self.quote: str = str(quote)
        self.movie_url: str =str(movie_url)
header = { 'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0'}

movies_info:[MovieInfo] = []

# 每页25个个元素 总共10页
for i in range(0,250,25):
    # Step 1: 获取数据
    url = f'https://movie.douban.com/top250?start={i}&filter='
    
    response = requests.get(url,headers=header)
    html_content = response.text
    
    # Step 2: 使用BeautifulSoup解析
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Step 3: 提取最小父级标签 
    # 这里应该是item而不是info
    # 因为class为info的节点中没有影片的序号信息 
    # 序号在class为item的div的另一个子节点下
    # 而class为item的div包含了所需的全部内容
    movie_items = soup.find_all('div', class_='item')
    
    success = False
    
    try:
        # Step 4: 提取一组信息
        movie_item = movie_items[0]
        success = True
    except:
        print('提取信息失败')
        continue
    
    # Step 5: 循环提取所有信息
    for movie_item in movie_items:
        # 电影序号
        rank = movie_item.find('em').text.strip()
        
        # 电影名称
        name = movie_item.find('span', class_='title').text.strip()
        
        # 评分
        rating = movie_item.find('span', class_='rating_num').text.strip()
        
        # 推荐语
        quote = movie_item.find('span', class_='inq')
        quote_text = quote.text.strip() if quote else "暂无推荐语"
        
        # 电影链接
        movie_url = movie_item.find('a')['href']
        movie_info:MovieInfo = MovieInfo(rank,name,rating,quote_text,movie_url)
        movies_info.append(movie_info)
        
        
        
# 打印结果
for movie_info in movies_info:
    print("序号:", movie_info.rank)
    print("电影名称:", movie_info.name)
    print("评分:", movie_info.rating)
    print("推荐语:", movie_info.quote)
    print("链接:", movie_info.movie_url)
    print("\n")

序号: 1
电影名称: 肖申克的救赎
评分: 9.7
推荐语: 希望让人自由。
链接: https://movie.douban.com/subject/1292052/


序号: 2
电影名称: 霸王别姬
评分: 9.6
推荐语: 风华绝代。
链接: https://movie.douban.com/subject/1291546/


序号: 3
电影名称: 阿甘正传
评分: 9.5
推荐语: 一部美国近现代史。
链接: https://movie.douban.com/subject/1292720/


序号: 4
电影名称: 泰坦尼克号
评分: 9.5
推荐语: 失去的才是永恒的。
链接: https://movie.douban.com/subject/1292722/


序号: 5
电影名称: 千与千寻
评分: 9.4
推荐语: 最好的宫崎骏，最好的久石让。
链接: https://movie.douban.com/subject/1291561/


序号: 6
电影名称: 这个杀手不太冷
评分: 9.4
推荐语: 怪蜀黍和小萝莉不得不说的故事。
链接: https://movie.douban.com/subject/1295644/


序号: 7
电影名称: 美丽人生
评分: 9.5
推荐语: 最美的谎言。
链接: https://movie.douban.com/subject/1292063/


序号: 8
电影名称: 星际穿越
评分: 9.4
推荐语: 爱是一种力量，让我们超越时空感知它的存在。
链接: https://movie.douban.com/subject/1889243/


序号: 9
电影名称: 盗梦空间
评分: 9.4
推荐语: 诺兰给了我们一场无法盗取的梦。
链接: https://movie.douban.com/subject/3541415/


序号: 10
电影名称: 楚门的世界
评分: 9.4
推荐语: 如果再也不能见到你，祝你早安，午安，晚安。
链接: https://movie.douban.com/subject/1292064/


序号: 11
电影名称: 辛德勒的名单
评分: 9.5
推荐语: 拯救一个人，就是拯救整个世界。
链接: https://movie.douban.com/su



【实验8-5】目标：网易云音乐或者酷狗音乐或者QQ音乐平台爬取音乐评论

(1)查找并赋值请求头headers
(2)查找评论内容、用户名、评论的时间
(3)将载荷中的表单内容赋值给params
(4)实验操作可以借鉴学习网址：
python爬取网易云音乐评论：https://www.jianshu.com/p/94252ab4fbf5
(5)将爬取的评论输出

In [116]:
import requests
import json
import time

# 请求头
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0',
    'Referer': 'https://music.163.com/',
    'Host': 'music.163.com'
}

# 获取评论
def get_comments(song_id, page=1):
    url = 'https://music.163.com/api/v1/resource/comments/R_SO_4_{}?limit=20&offset={}'.format(song_id, (page-1)*20)
    params = {
        'rid': 'R_SO_4_{}'.format(song_id),
        'threadId': 'R_SO_4_{}'.format(song_id),
        'offset': (page-1)*20,
        'limit': 20,
        'csrf_token': ''
    }
    response = requests.get(url, headers=headers, params=params)
    data = json.loads(response.text)
    comments = data['comments']
    return comments

# 解析评论
def parse_comments(comments):
    parsed_comments = []
    for comment in comments:
        content = comment['content']
        nickname = comment['user']['nickname']
        comment_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(comment['time']/1000))
        parsed_comments.append({
            'nickname': nickname,
            'content': content,
            'time': comment_time
        })
    return parsed_comments

# 主函数
def main(song_id):
    comments = []
    page = 1
    while True:
        print('正在爬取第{}页评论...'.format(page))
        current_comments = get_comments(song_id, page)
        if not current_comments:
            break
        parsed_comments = parse_comments(current_comments)
        comments.extend(parsed_comments)
        page += 1
    
    print('共爬取到{}条评论'.format(len(comments)))
    for comment in comments:
        print('用户名:', comment['nickname'])
        print('评论时间:', comment['time'])
        print('评论内容:', comment['content'])
        print('----------------------')

if __name__ == "__main__":
    song_id = input("请输入歌曲ID(可通过网易云歌曲网站链接末尾查看)：")
    main(song_id)


正在爬取第1页评论...


KeyError: 'comments'